In [2]:
from datetime import datetime, timedelta
import pandas as pd

import pybcm
from matplotlib import pyplot as plt
from pybcm.config import BCMConfig
from pybcm.rest import RestClient
from pybcm.brick_data import BrickData
from pybcm.const import *

config = BCMConfig('../config/bcm.ini')
rc = RestClient(config)
bd = BrickData(config)

expire = timedelta(days=15)

Initializing <class 'pybcm.config.BCMConfig'> ../config/bcm.ini


In [51]:
from datetime import datetime, timedelta

ts = datetime.now()
te = datetime.now()
delta = te - ts
print(f"{delta.microseconds/1000}ms")

0.073ms


In [64]:
from datetime import time, timedelta

def timeit(method):
    def timed(*args, **kw):
        ts = datetime.now()
        result = method(*args, **kw)
        te = datetime.now()
        delta = te - ts
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = delta.seconds
        else:
            print(f"{method.__name__}, {delta.microseconds/1000:.3f} ms")     
        return result
    return timed

In [68]:
@timeit
def testit():
    print("some stuff")
    
testit()
testit()
testit()

some stuff
testit, 0.080 ms
some stuff
testit, 0.034 ms
some stuff
testit, 0.033 ms


In [95]:
expire = timedelta(days=15)

def memoize_bd(function):
    """Used for wrapping functions in the BrickData class that take the arguments
        itemid, colorid, new_or_used=new_or_used, guide_type=guide_type
    """
    store_file = '/Users/Keith/Projects/pybcm_proj/resources/cache.hd5'

    def wrapper(itemid, colorid, new_or_used=None, guide_type=None, **kwargs):
        
        key = '_'.join([function.__name__, itemid, colorid, new_or_used, guide_type])
        with pd.HDFStore(store_file) as store:
            if key in store:
            # if key in hd5 and it's not too old
                print('Retrieving df from Store')
                df = store.get(key,)
                return df
            # return hd5 df
            else:
            # call the function and cache
                print('Calling function')
                df = function(itemid, colorid, new_or_used=new_or_used, guide_type=guide_type)
                store.put(key, df, format='table')
                return df

    return wrapper

In [97]:
@timeit
@memoize_bd
def get_price(itemid, colorid, new_or_used='N', guide_type='stock'):
    return bd.get_part_price_summary(itemid, colorid, new_or_used=new_or_used, guide_type=guide_type)

get_price('2420', '11', new_or_used='N', guide_type='stock')
get_price('2420', '12', new_or_used='N', guide_type='stock')

Retrieving df from Store
wrapper, 16.367 ms
Calling function
wrapper, 892.480 ms


,item_id,itemtype,color_id,new_or_used,currency_code,avg_price,max_price,min_price,qty_avg_price,unit_quantity,total_quantity,sold_or_stock,region
0,2420,PART,12,N,USD,0.1156,6.9201,0.018,0.0987,9095,653409,stock,north_america


In [98]:
store_file = '/Users/Keith/Projects/pybcm_proj/resources/cache.hd5'

with pd.HDFStore(store_file) as store:
    print(store.keys())

['/2420_86_N_stock', '/2420|86|N|stock', '/_242086Nstock', '/_2420_10_N_stock', '/_2420_11_N_stock', '/_2420_86_N_stock', '/__2420_86_N_stock', '/___main___2420_10_N_stock', '/___main___2420_11_N_stock', '/_get_price_2420_10_N_stock', '/_get_price_2420_11_N_stock', '/_|2420|86|N|stock', '/get_price_2420_10_N_stock', '/get_price_2420_11_N_stock', '/get_price_2420_12_N_stock']


In [116]:
import shelve

In [119]:
def memoize_rc(function):
    """Used for wrapping functions in the RestClient class that take the arguments
        itemid, itemtypeid, colorid, new_or_used, guide_type, vat, region
    """
    shelf_file = '/Users/Keith/Projects/pybcm_proj/resources/rccache'

    def wrapper(*args, **kwargs):
        
        key = '_'.join([function.__name__, *args, *kwargs])
        print(key)
        with shelve.open(shelf_file) as shelf:
            if key in shelf:
            # if key in shelf and it's not too old
                print('Retrieving df from Store')
                data = shelf[key]
                return data
            
            else:
            # call the function and cache
                print('Calling function')
                data = function(*args, **kwargs)
                shelf[key] = data
                return data

    return wrapper

In [120]:
@timeit
@memoize_rc
def get_item(itemid, itemtypeid):
    return rc.get_item(itemid, itemtypeid)

get_item('2420', 'PART')
get_item('2420', 'PART')

get_item_2420_PART
Calling function
wrapper, 611.714 ms
get_item_2420_PART
Retrieving df from Store
wrapper, 1.338 ms


{'no': '2420',
 'name': 'Plate 2 x 2 Corner',
 'type': 'PART',
 'category_id': 26,
 'image_url': '//img.bricklink.com/PL/2420.jpg',
 'thumbnail_url': '//img.bricklink.com/P/4/2420.jpg',
 'weight': '0.54',
 'dim_x': '2.00',
 'dim_y': '2.00',
 'dim_z': '0.00',
 'year_released': 1987,
 'is_obsolete': False}

In [122]:
@timeit
@memoize_rc
def get_subsets(itemid, itemtype):
    return rc.get_subsets(itemid, itemtype)

get_subsets('9492-1', 'SET')
get_subsets('9492-1', 'SET')

get_subsets_9492-1_SET
Retrieving df from Store
wrapper, 3.518 ms
get_subsets_9492-1_SET
Retrieving df from Store
wrapper, 2.307 ms


[{'match_no': 0,
  'entries': [{'item': {'no': '2357',
     'name': 'Brick 2 x 2 Corner',
     'type': 'PART',
     'category_id': 5},
    'color_id': 86,
    'quantity': 2,
    'extra_quantity': 0,
    'is_alternate': False,
    'is_counterpart': False}]},
 {'match_no': 0,
  'entries': [{'item': {'no': '2420',
     'name': 'Plate 2 x 2 Corner',
     'type': 'PART',
     'category_id': 26},
    'color_id': 85,
    'quantity': 4,
    'extra_quantity': 0,
    'is_alternate': False,
    'is_counterpart': False}]},
 {'match_no': 0,
  'entries': [{'item': {'no': '2431',
     'name': 'Tile 1 x 4',
     'type': 'PART',
     'category_id': 37},
    'color_id': 86,
    'quantity': 2,
    'extra_quantity': 0,
    'is_alternate': False,
    'is_counterpart': False}]},
 {'match_no': 0,
  'entries': [{'item': {'no': '2432',
     'name': 'Tile, Modified 1 x 2 with Handle',
     'type': 'PART',
     'category_id': 38},
    'color_id': 11,
    'quantity': 1,
    'extra_quantity': 0,
    'is_alternate'

In [136]:

def try_args(*args, **kwargs):
    key = '_'.join([ *args, *kwargs])
    print(f"{', '.join([*args, *kwargs.values()])}")
    print(key)
try_args('argw1', 'arg2', arg3='arg3text')


argw1, arg2, arg3text
argw1_arg2_arg3


In [130]:
rc.get_price_guide('3006', 'PART', '10', new_or_used = 'U', guide_type='stock')

{'item': {'no': '3006', 'type': 'PART'},
 'new_or_used': 'U',
 'currency_code': 'USD',
 'min_price': '0.0100',
 'max_price': '28.5700',
 'avg_price': '0.5648',
 'qty_avg_price': '0.6943',
 'unit_quantity': 1763,
 'total_quantity': 8210,
 'price_detail': [{'quantity': 134,
   'unit_price': '0.5000',
   'shipping_available': True,
   'qunatity': 134},
  {'quantity': 109,
   'unit_price': '0.5000',
   'shipping_available': True,
   'qunatity': 109},
  {'quantity': 97,
   'unit_price': '0.7500',
   'shipping_available': True,
   'qunatity': 97},
  {'quantity': 93,
   'unit_price': '1.8525',
   'shipping_available': True,
   'qunatity': 93},
  {'quantity': 75,
   'unit_price': '0.2610',
   'shipping_available': True,
   'qunatity': 75},
  {'quantity': 74,
   'unit_price': '0.8320',
   'shipping_available': True,
   'qunatity': 74},
  {'quantity': 64,
   'unit_price': '1.1000',
   'shipping_available': True,
   'qunatity': 64},
  {'quantity': 62,
   'unit_price': '1.8525',
   'shipping_avail